In [1]:
import re
import pyodbc
import pandas as pd
import datetime
import collections

In [28]:
regex = r'(?<=^|(?<=[^a-zA-Z0-9-_\./]))@([A-Za-z]+[A-Za-z0-9-_.]+[A-Za-z0-9-_])'

In [3]:
with open('pw.txt') as f:
    for l in f:
        uname, pw = l.split()

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=vip.hivesql.io;"
            "Database=DBHive;"
            f"UID={uname};"
            f"PWD={pw};")
cnxn = pyodbc.connect(cnxn_str)

In [32]:
query="""select c.title , c.created ,c.body 
from
    Tags
    INNER JOIN Comments c ON Tags.comment_id = C.ID
WHERE
    Tags.tag = 'followfriday'
    and c.author = 'steevc'
    and  c.[depth] =0
    --and c.created > '2021-01-01'
order by c.created """

In [36]:
r = pd.read_sql(query, cnxn)
users = []
for i, l in r.iterrows():
    for n in re.finditer(regex, l['body']):
        users.append(n.group().lower())
#collections.Counter(users)

In [53]:
print('''|User|Mentions|Last Post|
|-|-|-|''')
for u in collections.Counter(users).most_common(50):
    n = u[0][1:]
    r = pd.read_sql(f"select last_post from accounts where name='{n}'", cnxn)
    lp = r.loc[0].at['last_post'].strftime('%d-%b-%Y')
    print(f'|{u[0]}|{u[1]}|{lp}|')
accounts.last_post

|User|Mentions|Last Post|
|-|-|-|
|@coingecko|67|27-Jan-2021|
|@slobberchops|40|19-Nov-2021|
|@katharsisdrill|38|04-Oct-2021|
|@steemflagrewards|33|25-Jul-2021|
|@pennsif|30|26-Oct-2021|
|@dickturpin|29|19-Nov-2021|
|@tenkminnows|29|26-Oct-2021|
|@the-bitcoin-dood|27|19-Nov-2021|
|@arseniclullaby|24|03-Nov-2021|
|@lloyddavis|23|15-Nov-2021|
|@detlev|21|18-Nov-2021|
|@splinterlands|21|18-Nov-2021|
|@sidekickmatt|20|18-Nov-2021|
|@northwestnomad|19|18-Nov-2021|
|@stateofthedapps|19|26-Mar-2020|
|@redrica|17|25-Dec-2020|
|@nathanmars|16|19-Nov-2021|
|@exhaust|16|18-Nov-2021|
|@uwelang|15|17-Nov-2021|
|@rebeccabe|15|01-Oct-2019|
|@luzcypher|15|14-Sep-2020|
|@papilloncharity|15|19-Nov-2021|
|@drwatson|15|12-Nov-2021|
|@silviabeneforti|14|19-Nov-2021|
|@paolobeneforti|14|15-Nov-2021|
|@dreamrafa|14|11-Oct-2021|
|@ultravioletmag|14|19-Nov-2021|
|@soundlegion|13|14-Nov-2021|
|@stav|13|10-Nov-2021|
|@roelandp|13|14-Nov-2021|
|@steembasicincome|13|13-Nov-2021|
|@c0ff33a|13|18-Nov-2021|
|@risings

NameError: name 'accounts' is not defined

In [56]:
len(collections.Counter(users))

863